In [1]:
import pandas as pd
import numpy as np
import sqlite3
import folium
from folium.plugins import HeatMap,HeatMapWithTime

In [2]:
db_conn = sqlite3.connect("autobusy_i_tramwaje.sqlite")

In [ ]:
buses = pd.read_sql_query("SELECT * FROM buses WHERE time >= Datetime('now', '-270 seconds', 'localtime')", db_conn)
trams = pd.read_sql_query("SELECT * FROM trams WHERE time >= Datetime('now', '-270 seconds', 'localtime')", db_conn)

In [ ]:
buses

In [ ]:
trams

In [ ]:
# wyznaczamy środek mapy
map = folium.Map(location=(np.mean(list(trams.lat)+list(buses.lat)),
                          np.mean(list(trams.long)+list(buses.long))),
                 zoom_start=10)

# dodajemy autobusy
for i in range(len(buses)):
    folium.Circle(location=(buses.iloc[i]['lat'], buses.iloc[i]['long']),
                  tooltip="Autobus: "+buses.iloc[i]['line'],
                  radius=10,
                  color='red', fill_color='darkred',
                  fill_opacity=1,
                  fill=True).add_to(map)
    
# dodajemy tramwaje
for i in range(len(trams)):
    folium.Circle(location=(trams.iloc[i]['lat'], trams.iloc[i]['long']),
                  tooltip="Tramwaj: "+trams.iloc[i]['line'],
                  radius=10,
                  color='blue', fill_color='darkblue',
                  fill_opacity=1,
                  fill=True).add_to(map)
    
# pokazujemy mapę
map

In [3]:
# autobus konkternej linii
bus = pd.read_sql_query("SELECT * FROM buses WHERE line = 518", db_conn)

In [4]:
bus

,line,time,long,lat,brigade
0,518,2019-12-05 09:12:57,21.045197,52.264580,011
1,518,2019-12-05 11:27:31,21.007755,52.248032,9
2,518,2019-12-05 11:27:33,21.005484,52.248947,2
3,518,2019-12-05 11:27:30,20.954636,52.315166,7
4,518,2019-12-05 11:27:32,20.942087,52.334442,6
...,...,...,...,...,...
24883,518,2019-12-06 18:16:51,20.941542,52.332653,6
24884,518,2019-12-06 18:16:46,20.999451,52.291054,3
24885,518,2019-12-06 18:16:47,20.939291,52.332302,4
24886,518,2019-12-06 18:16:49,20.989769,52.264851,2


In [ ]:
# która brygada występuje najczęściej?
brigade_number = bus.groupby('brigade').size().reset_index()
brigade_number.columns = ['brigade', 'n_times']
brigade_number = brigade_number[brigade_number['n_times'] == max(brigade_number['n_times'])].iloc[0]['brigade']

In [ ]:
# tylko ona nas interesuje na mapie
bus_bridage = bus[bus['brigade'] == brigade_number]

In [ ]:
bus_bridage

In [5]:
# wyznaczamy środek mapy
bus_map = folium.Map(location=(np.mean(bus.lat), np.mean(bus.long)), zoom_start=10)

In [ ]:
# dodajemy autobusy
for i in range(len(bus_bridage)):
    folium.Circle(location=(bus_bridage.iloc[i]['lat'], bus_bridage.iloc[i]['long']),
                  radius=10,
                  tooltip=bus_bridage.iloc[i]['time'] + " (Brygada: " + bus_bridage.iloc[i]['brigade'] + ")",
                  color='red', fill_color='darkred',
                  fill_opacity=1,
                  fill=True).add_to(bus_map)

# pokazujemy mapę
bus_map

In [6]:
# Generate heat map
HeatMap(data=zip(bus.lat, bus.long),
        radius=8,
        max_zoom=13).add_to(bus_map)

bus_map

In [7]:
all_trams = pd.read_sql_query("""
    SELECT long, lat, COUNT(*) AS c
    FROM trams
    GROUP BY long, lat
    """, db_conn)

In [8]:
map_data = all_trams[['lat', 'long', 'c']].groupby(['lat', 'long']).sum().reset_index()
map_data = map_data.values.tolist()

In [ ]:
# wyznaczamy środek mapy
all_trams_map = folium.Map(location=(np.mean(all_trams.lat), np.mean(all_trams.long)), zoom_start=10)

In [ ]:
# Generate heat map
HeatMap(data=map_data,
        radius=8,
        max_zoom=13).add_to(all_trams_map)

In [ ]:
all_trams_map

In [10]:
# to samo dla autobusów
all_buses = pd.read_sql_query("""
    SELECT long, lat, COUNT(*) AS c
    FROM 
    (
        SELECT
            ROUND(long, 3) AS long,
            ROUND(lat, 3) AS lat,
            time
        FROM buses
    )
    GROUP BY long, lat
    """, db_conn)

map_data = all_buses[['lat', 'long', 'c']].groupby(['lat', 'long']).sum().reset_index()
map_data = map_data.values.tolist()

# wyznaczamy środek mapy
all_buses_map = folium.Map(location=(np.mean(all_buses.lat), np.mean(all_buses.long)), zoom_start=10)

# Generate heat map
HeatMap(data=map_data,
        radius=8,
        max_zoom=13).add_to(all_buses_map)

all_buses_map

In [11]:
all_trams_time = pd.read_sql_query("""
    SELECT
        long, lat,
        datetime(strftime('%s', time) - strftime('%s', time) % 120, 'unixepoch', 'localtime') as time_round,
        COUNT(*) AS c
    FROM
    (
        SELECT
            ROUND(long, 3) AS long,
            ROUND(lat, 3) AS lat,
            time
        FROM buses
    )
    GROUP BY long, lat
    """, db_conn)

In [12]:
df_hour_list = []

for hour in all_trams_time.time_round.sort_values().unique():
    map_data_hour = all_trams_time.loc[all_trams_time.time_round == hour, ['lat', 'long', 'c']]
    map_data_hour = map_data_hour.groupby(['lat', 'long']).sum().reset_index()
    map_data_hour = map_data_hour.values.tolist()
    df_hour_list.append(map_data_hour)

In [13]:
# wyznaczamy środek mapy
all_trams_map_time = folium.Map(location=(np.mean(all_trams_time.lat), np.mean(all_trams_time.long)), zoom_start=10)

In [14]:
HeatMapWithTime(data=df_hour_list,
               radius=8,
               use_local_extrema=True).add_to(all_trams_map_time)

In [15]:
all_trams_map_time